In [100]:
import torch
import numpy as np

In [113]:

#### Hyper parameters ####

num_arms = 10
num_agents = 10
max_epochs = 10000
epsilon = 0.25
influence_weight = 0.25
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

##########################


### Initialization ###

arm_performance = torch.rand(num_arms, 2, device=device)
history = torch.zeros(max_epochs, num_agents, num_arms, device=device)
adjacency_matrix = torch.rand(num_agents, num_agents, requires_grad=True, device=device)

######################

best_arm = torch.argmax(arm_performance[:, 0])
print(f"Best arm is {best_arm} ")

Best arm is 2 


In [114]:
for epoch in range(1, max_epochs):
# torch.sqrt(torch.log(epoch)/(1+))
    performance_estimate = history.sum(dim=0) / (history.sum(dim=0).sum(dim=1).unsqueeze(1) + 1)
    arm_pull_count = history.bool().sum(dim=0).int()
    choice = torch.argmax(performance_estimate + torch.sqrt(np.log(epoch)/(1+arm_pull_count)) + influence_weight * adjacency_matrix @ performance_estimate, dim=1)
    
    rewards = torch.normal(mean=arm_performance[choice, 0], std=torch.abs(arm_performance[choice, 1]))


    current_epoch_record = torch.zeros(num_agents, num_arms)

    for i in range(num_agents):
        for j in range(num_arms):
            if j == choice[i]:
                current_epoch_record[i, j] = rewards[i]


    history[epoch] = current_epoch_record
